# 07 - Carga a PostGIS

**Objetivo:** Cargar datos procesados a PostgreSQL/PostGIS para análisis espacial y backend de Streamlit

**Entradas:**
- CSV: `parroquias_con_petroleo.csv` (tabla final integrada)
- Shapefiles: parroquias, pozos, contaminación

**Salidas:**
- Base de datos PostgreSQL/PostGIS con:
  - Tabla `parroquias` (con geometría)
  - Tabla `pozos_petroleros`
  - Tabla `sitios_contaminacion`
  - Índices espaciales

---

## 1. Setup e Imports

In [3]:
# Celda 0: Verificar e instalar dependencias
import sys
print(f"🐍 Python usado por el kernel: {sys.executable}")
print(f"📦 Instalando librerías en este Python...\n")

!{sys.executable} -m pip install sqlalchemy psycopg2-binary geoalchemy2

🐍 Python usado por el kernel: /opt/homebrew/opt/python@3.10/bin/python3.10
📦 Instalando librerías en este Python...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 35.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
# Imports
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

print("✅ Librerías cargadas")

✅ Librerías cargadas


---
## 2. Configuración de Conexión PostgreSQL

In [2]:
# Configuración de conexión (AJUSTAR según tu setup)
DB_CONFIG = {
    'host': 'localhost',
    'port': 5434,
    'database': 'prototipo_salud',
    'user': 'postgres',
    'password': 'password'  # ← CAMBIAR ESTO
}

# Crear connection string
conn_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

print("🔧 Configuración de conexión:")
print(f"   Host: {DB_CONFIG['host']}")
print(f"   Puerto: {DB_CONFIG['port']}")
print(f"   Database: {DB_CONFIG['database']}")
print(f"   Usuario: {DB_CONFIG['user']}")

🔧 Configuración de conexión:
   Host: localhost
   Puerto: 5434
   Database: prototipo_salud
   Usuario: postgres


In [3]:
# Crear engine de SQLAlchemy
try:
    engine = create_engine(conn_string)
    
    # Test de conexión
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print("\n✅ CONEXIÓN EXITOSA")
        print(f"   PostgreSQL: {version.split(',')[0]}")
        
        # Verificar PostGIS
        result = conn.execute(text("SELECT PostGIS_Version();"))
        postgis_version = result.fetchone()[0]
        print(f"   PostGIS: {postgis_version}")
        
except Exception as e:
    print(f"\n❌ ERROR DE CONEXIÓN: {e}")
    print("\n💡 SOLUCIONES:")
    print("   1. Verifica que el contenedor Docker esté corriendo: docker ps")
    print("   2. Verifica el password en DB_CONFIG")
    print("   3. Verifica el puerto (5432 por defecto)")


✅ CONEXIÓN EXITOSA
   PostgreSQL: PostgreSQL 15.4 (Debian 15.4-1.pgdg110+1) on x86_64-pc-linux-gnu
   PostGIS: 3.3 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


---
## 3. Rutas de Datos

In [4]:
# Rutas
BASE_DIR = Path('../data')
PROCESSED_DIR = BASE_DIR / 'processed'
RAW_CONALI = BASE_DIR / 'raw' / 'conali_limites'
RAW_PETROLEO = BASE_DIR / 'raw' / 'maate_petroleo'

# Archivos
CSV_PARROQUIAS = PROCESSED_DIR / 'parroquias_con_petroleo.csv'
SHP_PARROQUIAS = RAW_CONALI / 'LIMITE_PARROQUIAL_CONALI_CNE_2022.shp'
SHP_POZOS = RAW_PETROLEO / 'Pozos.shp'
SHP_CONTAMINACION = RAW_PETROLEO / 'v_aa011_contaminacion_hidrocarburo_pMPoint.shp'

print("📁 Rutas configuradas:")
print(f"   CSV parroquias: {CSV_PARROQUIAS.exists()}")
print(f"   SHP parroquias: {SHP_PARROQUIAS.exists()}")
print(f"   SHP pozos: {SHP_POZOS.exists()}")
print(f"   SHP contaminación: {SHP_CONTAMINACION.exists()}")

📁 Rutas configuradas:
   CSV parroquias: True
   SHP parroquias: True
   SHP pozos: True
   SHP contaminación: True


---
## 4. Cargar Datos

In [5]:
# 1. Cargar CSV con datos procesados
print("📂 CARGANDO DATOS PROCESADOS\n")

df_parroquias = pd.read_csv(CSV_PARROQUIAS)
print(f"✅ CSV parroquias: {len(df_parroquias)} registros")
print(f"   Columnas: {len(df_parroquias.columns)}")

📂 CARGANDO DATOS PROCESADOS

✅ CSV parroquias: 1236 registros
   Columnas: 18


In [6]:
# 2. Cargar shapefile de parroquias (para geometrías)
gdf_parroquias_geom = gpd.read_file(SHP_PARROQUIAS)
print(f"\n✅ SHP parroquias (geometría): {len(gdf_parroquias_geom)} registros")
print(f"   CRS: {gdf_parroquias_geom.crs}")
print(f"   Columna código: DPA_PARROQ")


✅ SHP parroquias (geometría): 1236 registros
   CRS: EPSG:32717
   Columna código: DPA_PARROQ


In [7]:
# 3. Cargar shapefiles de petróleo
gdf_pozos = gpd.read_file(SHP_POZOS)
print(f"\n✅ SHP pozos: {len(gdf_pozos)} registros")

gdf_contaminacion = gpd.read_file(SHP_CONTAMINACION)
print(f"✅ SHP contaminación: {len(gdf_contaminacion)} registros")


✅ SHP pozos: 6287 registros
✅ SHP contaminación: 7850 registros


---
## 5. Preparar Tabla de Parroquias (CSV + Geometría)

In [8]:
# Merge CSV (datos procesados) con SHP (geometrías)
print("🔗 INTEGRANDO CSV CON GEOMETRÍAS\n")

# Seleccionar solo geometría del shapefile
geometrias = gdf_parroquias_geom[['CODPAR', 'geometry']].copy()

# 🔧 CONVERSIÓN DE TIPOS (FIX)
print("🔧 Convirtiendo tipos de datos...")
df_parroquias['codigo_dpa'] = df_parroquias['codigo_dpa'].astype(str).str.zfill(6)
geometrias['CODPAR'] = geometrias['CODPAR'].astype(str).str.zfill(6)

print(f"   CSV codigo_dpa tipo: {df_parroquias['codigo_dpa'].dtype}")
print(f"   SHP CODPAR tipo: {geometrias['CODPAR'].dtype}")
print(f"   Ejemplo CSV: {df_parroquias['codigo_dpa'].iloc[0]}")
print(f"   Ejemplo SHP: {geometrias['CODPAR'].iloc[0]}\n")

# Merge con CSV
gdf_parroquias_final = df_parroquias.merge(
    geometrias,
    left_on='codigo_dpa',
    right_on='CODPAR',
    how='left'
)

# Convertir a GeoDataFrame
gdf_parroquias_final = gpd.GeoDataFrame(
    gdf_parroquias_final,
    geometry='geometry',
    crs=gdf_parroquias_geom.crs
)

# Limpiar columna redundante
gdf_parroquias_final.drop('CODPAR', axis=1, inplace=True, errors='ignore')

print(f"✅ GeoDataFrame final: {len(gdf_parroquias_final)} registros")
print(f"   Columnas: {len(gdf_parroquias_final.columns)}")
print(f"   Con geometría: {gdf_parroquias_final['geometry'].notna().sum()}")
print(f"   Sin geometría: {gdf_parroquias_final['geometry'].isna().sum()}")
print(f"   CRS: {gdf_parroquias_final.crs}")

🔗 INTEGRANDO CSV CON GEOMETRÍAS

🔧 Convirtiendo tipos de datos...
   CSV codigo_dpa tipo: object
   SHP CODPAR tipo: object
   Ejemplo CSV: 000285
   Ejemplo SHP: 000285

✅ GeoDataFrame final: 1236 registros
   Columnas: 19
   Con geometría: 1236
   Sin geometría: 0
   CRS: EPSG:32717


---
## 6. Reprojectar a EPSG:4326 (WGS84)

Estándar para web mapping y Streamlit

In [9]:
print("🌍 REPROYECTANDO A WGS84 (EPSG:4326)\n")

# Reprojectar parroquias
if gdf_parroquias_final.crs != 'EPSG:4326':
    gdf_parroquias_final = gdf_parroquias_final.to_crs('EPSG:4326')
    print(f"✅ Parroquias reproyectadas: {gdf_parroquias_final.crs}")
else:
    print("✅ Parroquias ya están en EPSG:4326")

# Reprojectar pozos
if gdf_pozos.crs != 'EPSG:4326':
    gdf_pozos = gdf_pozos.to_crs('EPSG:4326')
    print(f"✅ Pozos reproyectados: {gdf_pozos.crs}")

# Reprojectar contaminación
if gdf_contaminacion.crs != 'EPSG:4326':
    gdf_contaminacion = gdf_contaminacion.to_crs('EPSG:4326')
    print(f"✅ Contaminación reproyectada: {gdf_contaminacion.crs}")

🌍 REPROYECTANDO A WGS84 (EPSG:4326)

✅ Parroquias reproyectadas: EPSG:4326
✅ Contaminación reproyectada: EPSG:4326


---
## 7. Cargar a PostGIS

In [10]:
print("📤 CARGANDO DATOS A POSTGIS\n")
print("="*70)

# 1. Tabla de parroquias (la principal)
print("\n1️⃣ Cargando tabla 'parroquias'...")
gdf_parroquias_final.to_postgis(
    name='parroquias',
    con=engine,
    if_exists='replace',  # Sobrescribe si ya existe
    index=False
)
print(f"   ✅ {len(gdf_parroquias_final)} registros cargados")

# 2. Tabla de pozos
print("\n2️⃣ Cargando tabla 'pozos_petroleros'...")
gdf_pozos.to_postgis(
    name='pozos_petroleros',
    con=engine,
    if_exists='replace',
    index=False
)
print(f"   ✅ {len(gdf_pozos)} registros cargados")

# 3. Tabla de contaminación
print("\n3️⃣ Cargando tabla 'sitios_contaminacion'...")
gdf_contaminacion.to_postgis(
    name='sitios_contaminacion',
    con=engine,
    if_exists='replace',
    index=False
)
print(f"   ✅ {len(gdf_contaminacion)} registros cargados")

print("\n" + "="*70)
print("🎉 CARGA COMPLETADA")

📤 CARGANDO DATOS A POSTGIS


1️⃣ Cargando tabla 'parroquias'...
   ✅ 1236 registros cargados

2️⃣ Cargando tabla 'pozos_petroleros'...
   ✅ 6287 registros cargados

3️⃣ Cargando tabla 'sitios_contaminacion'...
   ✅ 7850 registros cargados

🎉 CARGA COMPLETADA


---
## 8. Crear Índices Espaciales

In [11]:
print("🔧 CREANDO ÍNDICES ESPACIALES (GIST)\n")
print("="*70)

with engine.connect() as conn:
    # Índice en parroquias
    try:
        conn.execute(text("CREATE INDEX idx_parroquias_geom ON parroquias USING GIST (geometry);"))
        conn.commit()
        print("✅ Índice creado: parroquias.geometry")
    except Exception as e:
        print(f"⚠️ Índice ya existe o error: {e}")
    
    # Índice en pozos
    try:
        conn.execute(text("CREATE INDEX idx_pozos_geom ON pozos_petroleros USING GIST (geometry);"))
        conn.commit()
        print("✅ Índice creado: pozos_petroleros.geometry")
    except Exception as e:
        print(f"⚠️ Índice ya existe o error: {e}")
    
    # Índice en contaminación
    try:
        conn.execute(text("CREATE INDEX idx_contaminacion_geom ON sitios_contaminacion USING GIST (geometry);"))
        conn.commit()
        print("✅ Índice creado: sitios_contaminacion.geometry")
    except Exception as e:
        print(f"⚠️ Índice ya existe o error: {e}")

print("\n" + "="*70)
print("🎉 ÍNDICES CREADOS")

🔧 CREANDO ÍNDICES ESPACIALES (GIST)

✅ Índice creado: parroquias.geometry
✅ Índice creado: pozos_petroleros.geometry
✅ Índice creado: sitios_contaminacion.geometry

🎉 ÍNDICES CREADOS


---
## 9. Validaciones

In [12]:
print("✅ VALIDACIONES\n")
print("="*70)

with engine.connect() as conn:
    # 1. Contar registros
    print("\n📊 CONTEO DE REGISTROS:")
    
    result = conn.execute(text("SELECT COUNT(*) FROM parroquias;"))
    count_parroquias = result.fetchone()[0]
    print(f"   Parroquias: {count_parroquias}")
    
    result = conn.execute(text("SELECT COUNT(*) FROM pozos_petroleros;"))
    count_pozos = result.fetchone()[0]
    print(f"   Pozos: {count_pozos}")
    
    result = conn.execute(text("SELECT COUNT(*) FROM sitios_contaminacion;"))
    count_contaminacion = result.fetchone()[0]
    print(f"   Sitios contaminación: {count_contaminacion}")
    
    # 2. Verificar geometrías válidas
    print("\n🗺️ VALIDACIÓN DE GEOMETRÍAS:")
    
    result = conn.execute(text("""
        SELECT COUNT(*) 
        FROM parroquias 
        WHERE geometry IS NOT NULL AND ST_IsValid(geometry);
    """))
    valid_geoms = result.fetchone()[0]
    print(f"   Parroquias con geometría válida: {valid_geoms}/{count_parroquias}")
    
    # 3. Verificar CRS
    result = conn.execute(text("""
        SELECT DISTINCT ST_SRID(geometry) 
        FROM parroquias 
        WHERE geometry IS NOT NULL;
    """))
    srid = result.fetchone()[0]
    print(f"   SRID (debe ser 4326): {srid}")
    
    # 4. Query de ejemplo (parroquias con más pozos)
    print("\n🔝 TOP 5 PARROQUIAS CON MÁS POZOS:")
    result = conn.execute(text("""
        SELECT 
            p.nombre_parroquia,
            p.nombre_provincia,
            COUNT(pz.*) as num_pozos
        FROM parroquias p
        JOIN pozos_petroleros pz ON ST_Within(pz.geometry, p.geometry)
        GROUP BY p.nombre_parroquia, p.nombre_provincia
        ORDER BY num_pozos DESC
        LIMIT 5;
    """))
    
    for row in result:
        print(f"   • {row[0]} ({row[1]}): {row[2]} pozos")

print("\n" + "="*70)
print("✅ VALIDACIONES COMPLETADAS")

✅ VALIDACIONES


📊 CONTEO DE REGISTROS:
   Parroquias: 1236
   Pozos: 6287
   Sitios contaminación: 7850

🗺️ VALIDACIÓN DE GEOMETRÍAS:
   Parroquias con geometría válida: 1236/1236
   SRID (debe ser 4326): 4326

🔝 TOP 5 PARROQUIAS CON MÁS POZOS:
   • SAN JOSE DE ANCON (SANTA ELENA): 1363 pozos
   • ATAHUALPA (SANTA ELENA): 853 pozos
   • TARAPOA (SUCUMBIOS): 310 pozos
   • DAYUMA (ORELLANA): 268 pozos
   • LA LIBERTAD (SANTA ELENA): 232 pozos

✅ VALIDACIONES COMPLETADAS


---
## 10. Queries de Ejemplo

In [13]:
print("🔥 CORROBORANDO ANÁLISIS DEL NOTEBOOK 06: PARADOJA EXTRACTIVISTA\n")
print("="*70)

with engine.connect() as conn:
    # Query: Top 5 parroquias por INFRAESTRUCTURA TOTAL (pozos + contaminación)
    print("\n📍 TOP 5 PARROQUIAS MÁS PETROLERAS (Infraestructura Total):\n")
    
    result = conn.execute(text("""
        SELECT 
            p.nombre_parroquia,
            p.nombre_canton,
            p.nombre_provincia,
            p.num_infraestructura_petrolera as infra_total,
            p.num_pozos,
            p.num_sitios_contaminados,
            p.establecimientos_por_10k_hab as salud_10k,
            p.pct_poblacion_afro as pct_afro,
            p.poblacion_total,
            ROUND(CAST(p.densidad_petroleo_km2 AS NUMERIC), 2) as densidad_km2
        FROM parroquias p
        WHERE p.num_infraestructura_petrolera > 0
        ORDER BY p.num_infraestructura_petrolera DESC
        LIMIT 5;
    """))
    
    for i, row in enumerate(result, 1):
        print(f"{i}. {row[0]} ({row[1]}, {row[2]})")
        print(f"   • Infraestructura petrolera: {row[3]}")
        print(f"   • Pozos: {row[4]}")
        print(f"   • Sitios contaminados: {row[5]}")
        print(f"   • Densidad petrolera: {row[9]} infra/km²")
        print(f"   • Establecimientos salud/10k hab: {f'{row[6]:.2f}' if row[6] is not None else 'N/A'}")
        print(f"   • Población total: {row[8] if row[8] is not None else 'N/A'}")
        print(f"   • % Población afro: {f'{row[7]:.2f}' if row[7] is not None else 'N/A'}%")
        print()
    
    # Query: Correlación Petróleo vs Salud
    print("\n📊 CORRELACIÓN PETRÓLEO vs SALUD:\n")

    result = conn.execute(text("""
        SELECT 
            CASE 
                WHEN tiene_petroleo = 1 THEN 'Con petróleo'
                ELSE 'Sin petróleo'
            END as categoria,
            COUNT(*) as num_parroquias,
            ROUND(CAST(AVG(establecimientos_por_10k_hab) AS NUMERIC), 2) as salud_promedio,
            ROUND(CAST(AVG(num_infraestructura_petrolera) AS NUMERIC), 2) as infra_promedio
        FROM parroquias
        WHERE establecimientos_por_10k_hab IS NOT NULL
        GROUP BY tiene_petroleo
        ORDER BY tiene_petroleo DESC;
    """))

    for row in result:
        print(f"• {row[0]}:")
        print(f"  - Parroquias: {row[1]}")
        print(f"  - Salud promedio: {row[2]} establecimientos/10k hab")
        print(f"  - Infraestructura petrolera promedio: {row[3]}")
        print()
    
    # Query: Parroquias con CERO salud pero ALTA infraestructura petrolera
    print("\n🚨 PARROQUIAS CON PARADOJA EXTRACTIVISTA CRÍTICA:\n")
    print("   (Alta infraestructura petrolera + CERO acceso a salud)\n")
    
    result = conn.execute(text("""
        SELECT 
            p.nombre_parroquia,
            p.nombre_provincia,
            p.num_infraestructura_petrolera,
            p.num_pozos,
            p.num_sitios_contaminados,
            p.establecimientos_por_10k_hab,
            p.poblacion_total
        FROM parroquias p
        WHERE p.num_infraestructura_petrolera > 100
          AND (p.establecimientos_por_10k_hab = 0 OR p.establecimientos_por_10k_hab IS NULL)
        ORDER BY p.num_infraestructura_petrolera DESC
        LIMIT 10;
    """))
    
    count = 0
    for row in result:
        count += 1
        print(f"{count}. {row[0]} ({row[1]})")
        print(f"   • Infraestructura: {row[2]} | Pozos: {row[3]} | Contaminación: {row[4]}")
        print(f"   • Salud: {f'{row[5]:.2f}' if row[5] is not None else '0.00'}/10k hab | Población: {row[6]}")
        print()

print("="*70)
print("✅ ANÁLISIS COMPLETADO - Datos coinciden con Notebook 06")

🔥 CORROBORANDO ANÁLISIS DEL NOTEBOOK 06: PARADOJA EXTRACTIVISTA


📍 TOP 5 PARROQUIAS MÁS PETROLERAS (Infraestructura Total):

1. SAN JOSE DE ANCON (SANTA ELENA, SANTA ELENA)
   • Infraestructura petrolera: 1511
   • Pozos: 1363
   • Sitios contaminados: 148
   • Densidad petrolera: 22.98 infra/km²
   • Establecimientos salud/10k hab: N/A
   • Población total: N/A
   • % Población afro: N/A%

2. PACAYACU (LAGO AGRIO, SUCUMBIOS)
   • Infraestructura petrolera: 1234
   • Pozos: 172
   • Sitios contaminados: 1062
   • Densidad petrolera: 1.40 infra/km²
   • Establecimientos salud/10k hab: 2.37
   • Población total: 8428.0
   • % Población afro: 1.52%

3. SANSAHUARI (PUTUMAYO, SUCUMBIOS)
   • Infraestructura petrolera: 1177
   • Pozos: 121
   • Sitios contaminados: 1056
   • Densidad petrolera: 2.36 infra/km²
   • Establecimientos salud/10k hab: 0.00
   • Población total: 2669.0
   • % Población afro: 1.99%

4. SHUSHUFINDI CENTRAL (SHUSHUFINDI, SUCUMBIOS)
   • Infraestructura petrolera: 989

In [14]:
print("📊 QUERIES DE EJEMPLO\n")
print("="*70)

with engine.connect() as conn:
    # Query 1: Parroquias con alta población afro Y petróleo
    print("\n1️⃣ Parroquias con >5% población afro Y infraestructura petrolera:")
    result = conn.execute(text("""
        SELECT 
            nombre_parroquia,
            nombre_provincia,
            pct_poblacion_afro,
            num_infraestructura_petrolera
        FROM parroquias
        WHERE pct_poblacion_afro > 5 
          AND num_infraestructura_petrolera > 0
        ORDER BY pct_poblacion_afro DESC
        LIMIT 5;
    """))
    
    for row in result:
        print(f"   • {row[0]} ({row[1]}): {row[2]:.1f}% afro, {row[3]} infraestructura")
    
    # Query 2: Densidad de establecimientos de salud en zonas petroleras
    print("\n2️⃣ Densidad de salud promedio (tiene petróleo vs no tiene):")
    result = conn.execute(text("""
        SELECT 
            CASE WHEN tiene_petroleo = 1 THEN 'Con petróleo' ELSE 'Sin petróleo' END as categoria,
            AVG(establecimientos_por_10k_hab) as densidad_promedio,
            COUNT(*) as num_parroquias
        FROM parroquias
        WHERE establecimientos_por_10k_hab IS NOT NULL
        GROUP BY tiene_petroleo;
    """))
    
    for row in result:
        print(f"   • {row[0]}: {row[1]:.2f} establecimientos/10k hab ({row[2]} parroquias)")

print("\n" + "="*70)

📊 QUERIES DE EJEMPLO


1️⃣ Parroquias con >5% población afro Y infraestructura petrolera:
   • ROSA ZARATE (ESMERALDAS): 8.8% afro, 6 infraestructura
   • VICHE (ESMERALDAS): 7.5% afro, 2 infraestructura
   • TONCHIGUE (ESMERALDAS): 5.6% afro, 1 infraestructura
   • SHUSHUFINDI CENTRAL (SUCUMBIOS): 5.3% afro, 989 infraestructura

2️⃣ Densidad de salud promedio (tiene petróleo vs no tiene):
   • Sin petróleo: 7.77 establecimientos/10k hab (784 parroquias)
   • Con petróleo: 5.15 establecimientos/10k hab (63 parroquias)



---
## 11. Información de Tablas

In [15]:
print("📋 INFORMACIÓN DE TABLAS EN POSTGIS\n")
print("="*70)

with engine.connect() as conn:
    # Listar todas las tablas
    result = conn.execute(text("""
        SELECT 
            tablename,
            pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) AS size
        FROM pg_tables
        WHERE schemaname = 'public'
        ORDER BY pg_total_relation_size(schemaname||'.'||tablename) DESC;
    """))
    
    print("\n📊 TABLAS CARGADAS:")
    for row in result:
        print(f"   • {row[0]}: {row[1]}")
    
    # Info de geometry_columns (PostGIS)
    result = conn.execute(text("""
        SELECT 
            f_table_name,
            f_geometry_column,
            type,
            srid
        FROM geometry_columns
        WHERE f_table_schema = 'public';
    """))
    
    print("\n🗺️ COLUMNAS GEOMÉTRICAS:")
    for row in result:
        print(f"   • {row[0]}.{row[1]}: {row[2]} (SRID: {row[3]})")

print("\n" + "="*70)

📋 INFORMACIÓN DE TABLAS EN POSTGIS


📊 TABLAS CARGADAS:
   • parroquias: 224 MB
   • spatial_ref_sys: 7144 kB
   • sitios_contaminacion: 2000 kB
   • pozos_petroleros: 1864 kB

🗺️ COLUMNAS GEOMÉTRICAS:
   • sitios_contaminacion.geometry: MULTIPOINT (SRID: 4326)
   • parroquias.geometry: GEOMETRY (SRID: 4326)
   • pozos_petroleros.geometry: POINT (SRID: 4326)



---
## 12. Resumen Final

In [16]:
print("\n" + "="*70)
print("🎉 RESUMEN: CARGA A POSTGIS COMPLETADA")
print("="*70)

print("\n✅ TAREAS COMPLETADAS:")
print("  1. Conexión a PostgreSQL/PostGIS establecida")
print("  2. CSV integrado con geometrías de shapefile")
print("  3. Reproyección a EPSG:4326 (WGS84)")
print("  4. Tres tablas cargadas:")
print("     • parroquias (con todos los atributos)")
print("     • pozos_petroleros")
print("     • sitios_contaminacion")
print("  5. Índices espaciales GIST creados")
print("  6. Validaciones exitosas")

print("\n📊 ESTADÍSTICAS:")
print(f"  • Total parroquias: {count_parroquias}")
print(f"  • Total pozos: {count_pozos}")
print(f"  • Total sitios contaminación: {count_contaminacion}")
print(f"  • CRS: EPSG:{srid}")

print("\n🔗 CONEXIÓN:")
print(f"  Host: {DB_CONFIG['host']}")
print(f"  Puerto: {DB_CONFIG['port']}")
print(f"  Database: {DB_CONFIG['database']}")

print("\n💡 SIGUIENTE PASO:")
print("  Crear dashboard en Streamlit que consuma de PostGIS")

print("\n🔧 EJEMPLO DE CONEXIÓN EN STREAMLIT:")
print(f"  conn = psycopg2.connect(\"host={DB_CONFIG['host']} dbname={DB_CONFIG['database']} user={DB_CONFIG['user']} password=***\")")

print("\n" + "="*70)


🎉 RESUMEN: CARGA A POSTGIS COMPLETADA

✅ TAREAS COMPLETADAS:
  1. Conexión a PostgreSQL/PostGIS establecida
  2. CSV integrado con geometrías de shapefile
  3. Reproyección a EPSG:4326 (WGS84)
  4. Tres tablas cargadas:
     • parroquias (con todos los atributos)
     • pozos_petroleros
     • sitios_contaminacion
  5. Índices espaciales GIST creados
  6. Validaciones exitosas

📊 ESTADÍSTICAS:
  • Total parroquias: 1236
  • Total pozos: 6287
  • Total sitios contaminación: 7850
  • CRS: EPSG:4326

🔗 CONEXIÓN:
  Host: localhost
  Puerto: 5434
  Database: prototipo_salud

💡 SIGUIENTE PASO:
  Crear dashboard en Streamlit que consuma de PostGIS

🔧 EJEMPLO DE CONEXIÓN EN STREAMLIT:
  conn = psycopg2.connect("host=localhost dbname=prototipo_salud user=postgres password=***")

